In [1]:
import pandas as pd
df1 = pd.read_excel('../data/台塑企業_ 產品寶典20210303.xlsx',engine='openpyxl').iloc[:,:-1]
df2 = pd.read_excel('../data/寶典.v3.台塑網.20210901.xlsx',engine='openpyxl')
df2.columns = df1.columns
df = df1.append(df2)
df['品名'] = df['品名'].apply(lambda x:x.strip()) # 針對品名去除左右空白
品名2部門 = dict(zip(df['品名'],df['公司事業部門']))
品名2代號 = dict(zip(df['品名'],df['公司代號']))
NAME = 'POLYPROPYLENE RESIN GRADE'

In [2]:
品名2部門[NAME]

'塑聚丙烯部'

In [3]:
品名2代號[NAME]

'1P'

In [4]:
train_df = pd.read_csv('../data/preprocess_for_SQUAD_產品.csv')
train_df['Y_label'] = train_df['Y_label'].apply(lambda x:x.strip()) #針對SPEC去除左右空白
train_df[(train_df['Y_label'].str.contains(NAME))]

,Unnamed: 0,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2


In [5]:
# 根據品名從訓練資料搜索EXPNO,然後把EXPNO代入保典裡找公司部門
def find_department(x):
    return df.loc[df['公司代號']==train_df.loc[train_df.Y_label==x,'EXPNO'].dropna().value_counts().sort_values().index[-1],'公司事業部門'].value_counts().sort_values().index[-1]

In [6]:
find_department(NAME)

IndexError: index -1 is out of bounds for axis 0 with size 0

In [11]:
df[df['品名']==NAME]

,RIGID,分機,公司代號,公司事業部門,品名
222,工讀生,6136,1P,塑聚丙烯部,POLYPROPYLENE RESIN GRADE
658,工讀生,6136,1P,塑聚丙烯部,POLYPROPYLENE RESIN GRADE


In [10]:
train_df[train_df['Y_label']==NAME]

,Unnamed: 0,string_X_train,Y_label,EXPNO,from,string_Y_1,string_Y_2
